In [1]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.9 MB/s eta 0:00:00


In [2]:
!pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 35.8 MB/s eta 0:00:00


In [3]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.1 MB/s eta 0:00:00


# Work

In [81]:
from pdfminer.high_level import extract_text
import tabula

def extract_text_by_page(pdf_path):
    # Using extract_text from pdfminer.high_level to extract text page by page
    return [extract_text(pdf_path, page_numbers=[i]) for i in range(1, count_pages(pdf_path) + 1)]

def count_pages(pdf_path):
    # Use pdfminer to count the number of pages in the PDF
    from pdfminer.pdfpage import PDFPage
    with open(pdf_path, 'rb') as fh:
        return len(list(PDFPage.get_pages(fh)))

def find_pages_with_exact_text(pdf_path, search_texts):
    pages_with_exact_text = []
    for i, page_text in enumerate(extract_text_by_page(pdf_path), start=1):
        for search_text in search_texts:
          if search_text in page_text:  # Search for the exact phrase
            if i not in pages_with_exact_text:
              pages_with_exact_text.append(i)
    return pages_with_exact_text

pdf_path = '/content/RLI.pdf'  # Update this path to your PDF file
smoke_keywords = ['Male Non-Smoker','Female Non-Smoker', 'Male Smoker', 'Female Smoker']  # This is the exact phrase you're looking for
reinsurance_keywords = ['Reinsurance Premium Block']
face_band_keywords = ['Face Band']
# Find pages containing the exact phrase
pages_with_smoke_keywords = find_pages_with_exact_text(pdf_path, smoke_keywords)
pages_with_reinsurance_keywords = find_pages_with_exact_text(pdf_path, reinsurance_keywords)
pages_with_face_band_keywords = find_pages_with_exact_text(pdf_path, face_band_keywords)

# Smoke Data Tables Extraction

In [82]:
import pandas as pd
smoke_tables = []

for i in range(len(pages_with_smoke_keywords)):
    com_df = pd.DataFrame()
    # Extract tables from the current page
    tables = tabula.read_pdf(pdf_path, pages=pages_with_smoke_keywords[i]+1, multiple_tables=True)
    if len(tables)>1:
      for table in  tables:
        df1 = table
        new_headers = ['Issue age'] + [str(i) for i in range(1, len(df1.columns)-1)] + ['Ultimate']
        new_headers = new_headers[:len(df1.columns)]  # Limit to the actual number of columns
        original_first_row = df1.columns
        df1.columns = new_headers
        new_row = pd.DataFrame([original_first_row], columns=new_headers)
        new_row['Issue age'].iloc[0] = float(df1['Issue age'].iloc[0])
        df = pd.concat([new_row, df1], ignore_index=True)
        com_df = pd.concat([com_df, df], ignore_index=True)
      df = com_df
    else:
      df1 = tables[0]

      new_headers = ['Issue age'] + [str(i) for i in range(1, len(df1.columns)-1)] + ['Ultimate']
      new_headers = new_headers[:len(df1.columns)]  # Limit to the actual number of columns
      original_first_row = df1.columns
      df1.columns = new_headers
      new_row = pd.DataFrame([original_first_row], columns=new_headers)
      new_row['Issue age'].iloc[0] = float(df['Issue age'].iloc[0])
      df = pd.concat([new_row, df1], ignore_index=True)
    smoke_tables.append(df)

Mar 15, 2024 5:09:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:09:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:19 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:19 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:09:21 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:23 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:23 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:09:25 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:27 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:27 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [83]:
filename = 'TableData.xlsx'
with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
    # Write each DataFrame to a different sheet
    for i in range(len(smoke_tables)):
      smoke_tables[i].to_excel(writer, sheet_name=smoke_keywords[i], index=False)

# Reinsurance Data Table Extraction

In [84]:
reinsurance_tables_list = []

for page_number in pages_with_reinsurance_keywords:
    tables = tabula.read_pdf(pdf_path, pages=page_number + 1, multiple_tables=True)
    if len(tables)>0:
      reinsurance_tables_list.append(tables)

Mar 15, 2024 5:09:29 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:30 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:30 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:09:31 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:32 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:09:32 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:09:34 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:09:37 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:09:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:09:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:09:45 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:09:48 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:09:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFon

In [85]:
print(reinsurance_tables_list)

[[      Reinsurance   Policy Unnamed: 0 Total Policy     Quota Share Reinsured
0             NaN      NaN   Coverage          NaN                       NaN
1   Premium Block   Number        NaN         Face  (as a percentage of Face
2               1  4006112          1   10,000,000                       45%
3               1  4006504          1      150,000                       45%
4               1  4006541          1      100,000                       45%
5               1  4006604          1      250,000                       45%
6               1  4006652          1       50,000                       45%
7               1  4006748          1       50,000                       45%
8               1  4006753          1    3,000,000                       45%
9               1  4006832          1      100,000                       45%
10              1  4006844          1       50,000                       45%
11              1  4006874          1      150,000                       4

# Face Band Tabe Data Extraction

In [86]:
face_band_table_list = []

for page_number in pages_with_face_band_keywords:
    tables = tabula.read_pdf(pdf_path, pages=page_number + 1, multiple_tables=True)
    if len(tables)>0:
      face_band_table_list.append(tables)

Mar 15, 2024 5:10:34 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:10:35 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:10:35 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:10:37 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:10:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Mar 15, 2024 5:10:41 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:10:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 15, 2024 5:10:42 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [88]:
print(face_band_table_list)

[[      Unnamed: 0 Unnamed: 1 Unnamed: 2        Unnamed: 3 Unnamed: 4  \
0   Underwriting    Product     Gender        Risk Class   Duration   
1            All        All       Male      Preferred NT       1-20   
2            NaN        NaN        NaN               NaN      21-40   
3            NaN        NaN        NaN               NaN        41+   
4            NaN        NaN        NaN  Standard Plus NT       1-20   
5            NaN        NaN        NaN               NaN      21-40   
6            NaN        NaN        NaN               NaN        41+   
7            NaN        NaN        NaN       Standard NT       1-20   
8            NaN        NaN        NaN               NaN      21-40   
9            NaN        NaN        NaN               NaN        41+   
10           NaN        NaN        NaN  Standard Plus TB       1-20   
11           NaN        NaN        NaN               NaN      21-40   
12           NaN        NaN        NaN               NaN        41+   
13  